## Libraries

In [1]:
import numpy as np
import pandas as pd

## Remove NaN Comment

### Attractions

In [2]:
attraction_df, remove_list = pd.read_csv("attraction.csv", encoding = "utf-16"), list()

for i in range(attraction_df.shape[0]):
    if pd.isnull(attraction_df["Comments"][i]):
        remove_list.append(i)

attraction_df.drop(labels = remove_list, axis = 0, inplace = True)
attraction_df.to_csv("attraction_after.csv", encoding = "utf-16", index = False)

attraction_df

,Title,Comments,Url
0,竹湖,31 則評論,https://www.google.com/maps/place/%E7%AB%B9%E6...
1,寄梅亭,10 則評論,https://www.google.com/maps/place/%E5%AF%84%E6...
2,清華大學成功湖,379 則評論,https://www.google.com/maps/place/%E6%B8%85%E8...
3,靜心湖,"2,107 則評論",https://www.google.com/maps/place/%E9%9D%9C%E5...
4,沃夫帕克WolfPark 親子館(新竹親子餐廳/親子景點/室內親子),529 則評論,https://www.google.com/maps/place/%E6%B2%83%E5...
...,...,...,...
295,好好聚落,395 則評論,https://www.google.com/maps/place/%E5%A5%BD%E5...
296,宮原眼科,"24,400 則評論",https://www.google.com/maps/place/%E5%AE%AE%E5...
297,鳳鼻頭文化遺址,200 則評論,https://www.google.com/maps/place/%E9%B3%B3%E9...
298,高雄市忠烈祠,"1,896 則評論",https://www.google.com/maps/place/%E9%AB%98%E9...


### Restaurants

In [3]:
restaurant_df, remove_list = pd.read_csv("restaurant.csv", encoding = "utf-16"), list()

for i in range(restaurant_df.shape[0]):
    if pd.isnull(restaurant_df["Comments"][i]):
        remove_list.append(i)

restaurant_df.drop(labels = remove_list, axis = 0, inplace = True)
restaurant_df.to_csv("restaurant_after.csv", encoding = "utf-16", index = False)

restaurant_df

,Title,Comments,Url
0,LALA Kitchen 新美式餐廳 交大店,994 則評論,https://www.google.com/maps/place/LALA+Kitchen...
1,和選旅-硯宴軒,350 則評論,https://www.google.com/maps/place/%E5%92%8C%E9...
2,漢堡王Burger King 交大店,315 則評論,https://www.google.com/maps/place/%E6%BC%A2%E5...
3,RINGOAL車庫餐廳,383 則評論,https://www.google.com/maps/place/RINGOAL%E8%B...
4,橄欖樹廚房,669 則評論,https://www.google.com/maps/place/%E6%A9%84%E6...
...,...,...,...
291,夠麻吉股份有限公司,409 則評論,https://www.google.com/maps/place/%E5%A4%A0%E9...
292,紓咖,20 則評論,https://www.google.com/maps/place/%E7%B4%93%E5...
293,太魯閣晶英酒店梅園中餐廳,174 則評論,https://www.google.com/maps/place/%E5%A4%AA%E9...
294,CITYLINK 內湖店,"1,953 則評論",https://www.google.com/maps/place/CITYLINK+%E5...


### Bookstores

In [4]:
bookstore_df, remove_list = pd.read_csv("bookstore.csv", encoding = "utf-16"), list()

for i in range(bookstore_df.shape[0]):
    if pd.isnull(bookstore_df["Comments"][i]):
        remove_list.append(i)

bookstore_df.drop(labels = remove_list, axis = 0, inplace = True)
bookstore_df.to_csv("bookstore_after.csv", encoding = "utf-16", index = False)

bookstore_df

,Title,Comments,Url
0,麗文校園書局(交通大學店),68 則評論,https://www.google.com/maps/place/%E9%BA%97%E6...
1,水木書苑,763 則評論,https://www.google.com/maps/place/%E6%B0%B4%E6...
2,大學書局,"1,231 則評論",https://www.google.com/maps/place/%E5%A4%A7%E5...
3,筆耕小書店,86 則評論,https://www.google.com/maps/place/%E7%AD%86%E8...
4,校園書房(新竹勝利店),110 則評論,https://www.google.com/maps/place/%E6%A0%A1%E5...
...,...,...,...
209,書屋花甲Ｘ而立書店（台大店）,84 則評論,https://www.google.com/maps/place/%E6%9B%B8%E5...
210,太平青鳥,520 則評論,https://www.google.com/maps/place/%E5%A4%AA%E5...
211,書屋花甲,284 則評論,https://www.google.com/maps/place/%E6%9B%B8%E5...
213,練習曲書店,342 則評論,https://www.google.com/maps/place/%E7%B7%B4%E7...


### Departments

In [5]:
department_df, remove_list = pd.read_csv("department.csv", encoding = "utf-16"), list()

for i in range(department_df.shape[0]):
    if pd.isnull(department_df["Comments"][i]):
        remove_list.append(i)

department_df.drop(labels = remove_list, axis = 0, inplace = True)
department_df.to_csv("department_after.csv", encoding = "utf-16", index = False)

department_df

,Title,Comments,Url
0,大聯大百貨生活館(交大門市),56 則評論,https://www.google.com/maps/place/%E5%A4%A7%E8...
1,小雅10元百貨,79 則評論,https://www.google.com/maps/place/%E5%B0%8F%E9...
2,水木百貨,103 則評論,https://www.google.com/maps/place/%E6%B0%B4%E6...
3,小雅10元百貨,49 則評論,https://www.google.com/maps/place/%E5%B0%8F%E9...
4,大信五金百貨複合館,34 則評論,https://www.google.com/maps/place/%E5%A4%A7%E4...
...,...,...,...
198,德安百貨,"5,860 則評論",https://www.google.com/maps/place/%E5%BE%B7%E5...
199,DAISO大創百貨 台南德安店,479 則評論,https://www.google.com/maps/place/DAISO%E5%A4%...
200,漢神巨蛋購物廣場,"32,720 則評論",https://www.google.com/maps/place/%E6%BC%A2%E7...
201,漢神百貨,"8,876 則評論",https://www.google.com/maps/place/%E6%BC%A2%E7...


### Nightmarkets

In [6]:
nightmarket_df, remove_list = pd.read_csv("nightmarket.csv", encoding = "utf-16"), list()

for i in range(nightmarket_df.shape[0]):
    if pd.isnull(nightmarket_df["Comments"][i]):
        remove_list.append(i)

nightmarket_df.drop(labels = remove_list, axis = 0, inplace = True)
nightmarket_df.to_csv("nightmarket_after.csv", encoding = "utf-16", index = False)

nightmarket_df

,Title,Comments,Url
0,新竹慈雲夜市,"1,346 則評論",https://www.google.com/maps/place/%E6%96%B0%E7...
1,青草湖夜市,762 則評論,https://www.google.com/maps/place/%E9%9D%92%E8...
2,竹中夜市,73 則評論,https://www.google.com/maps/place/%E7%AB%B9%E4...
3,新竹市觀光夜市協會,2 則評論,https://www.google.com/maps/place/%E6%96%B0%E7...
4,新竹市城隍廟夜市,"9,105 則評論",https://www.google.com/maps/place/%E6%96%B0%E7...
...,...,...,...
113,新夜市鴨肉麵,51 則評論,https://www.google.com/maps/place/%E6%96%B0%E5...
114,嘉義文化路夜市,"22,108 則評論",https://www.google.com/maps/place/%E5%98%89%E7...
115,士林鹽水雞,66 則評論,https://www.google.com/maps/place/%E5%A3%AB%E6...
116,彌陀夜市,"2,228 則評論",https://www.google.com/maps/place/%E5%BD%8C%E9...


### Outlets

In [7]:
outlet_df, remove_list = pd.read_csv("outlet.csv", encoding = "utf-16"), list()

for i in range(outlet_df.shape[0]):
    if pd.isnull(outlet_df["Comments"][i]):
        remove_list.append(i)

outlet_df.drop(labels = remove_list, axis = 0, inplace = True)
outlet_df.to_csv("outlet_after.csv", encoding = "utf-16", index = False)

outlet_df

,Title,Comments,Url
0,Timberland 新竹愛買 Outlet,22 則評論,https://www.google.com/maps/place/Timberland+%...
2,GLORIA OUTLETS華泰名品城,"36,364 則評論",https://www.google.com/maps/place/GLORIA+OUTLE...
3,MITSUI OUTLET PARK 林口,"38,888 則評論",https://www.google.com/maps/place/MITSUI+OUTLE...
4,麗寶Outlet Mall,"21,671 則評論",https://www.google.com/maps/place/%E9%BA%97%E5...
5,UNDER ARMOUR 林口三井OUTLET店,"2,211 則評論",https://www.google.com/maps/place/UNDER+ARMOUR...
...,...,...,...
193,御殿場PREMIUM OUTLETS,"20,135 則評論",https://www.google.com/maps/place/%E5%BE%A1%E6...
194,三井OUTLET PARK 木更津,"12,904 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...
195,SHIATZY CHEN 夏姿陳,4 則評論,https://www.google.com/maps/place/SHIATZY+CHEN...
196,三井OUTLET PARK 幕張,"7,749 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...


## Remove Duplicate

### Attractions

In [8]:
attraction_after_df, records, remove_list = pd.read_csv("attraction_after.csv", encoding = "utf-16"), set(), list()

for i in range(attraction_after_df.shape[0]):
    if attraction_after_df["Title"][i] in records:
        remove_list.append(i)
    else:
        records.add(attraction_after_df["Title"][i])
        
attraction_after_df.drop(labels = remove_list, axis = 0, inplace = True)
attraction_after_df.to_csv("attraction_after.csv", encoding = "utf-16", index = False)

attraction_after_df

,Title,Comments,Url
0,竹湖,31 則評論,https://www.google.com/maps/place/%E7%AB%B9%E6...
1,寄梅亭,10 則評論,https://www.google.com/maps/place/%E5%AF%84%E6...
2,清華大學成功湖,379 則評論,https://www.google.com/maps/place/%E6%B8%85%E8...
3,靜心湖,"2,107 則評論",https://www.google.com/maps/place/%E9%9D%9C%E5...
4,沃夫帕克WolfPark 親子館(新竹親子餐廳/親子景點/室內親子),529 則評論,https://www.google.com/maps/place/%E6%B2%83%E5...
...,...,...,...
290,好好聚落,395 則評論,https://www.google.com/maps/place/%E5%A5%BD%E5...
291,宮原眼科,"24,400 則評論",https://www.google.com/maps/place/%E5%AE%AE%E5...
292,鳳鼻頭文化遺址,200 則評論,https://www.google.com/maps/place/%E9%B3%B3%E9...
293,高雄市忠烈祠,"1,896 則評論",https://www.google.com/maps/place/%E9%AB%98%E9...


### Restaurants

In [9]:
restaurant_after_df, records, remove_list = pd.read_csv("restaurant_after.csv", encoding = "utf-16"), set(), list()

for i in range(restaurant_after_df.shape[0]):
    if restaurant_after_df["Title"][i] in records:
        remove_list.append(i)
    else:
        records.add(restaurant_after_df["Title"][i])
        
restaurant_after_df.drop(labels = remove_list, axis = 0, inplace = True)
restaurant_after_df.to_csv("restaurant_after.csv", encoding = "utf-16", index = False)

restaurant_after_df

,Title,Comments,Url
0,LALA Kitchen 新美式餐廳 交大店,994 則評論,https://www.google.com/maps/place/LALA+Kitchen...
1,和選旅-硯宴軒,350 則評論,https://www.google.com/maps/place/%E5%92%8C%E9...
2,漢堡王Burger King 交大店,315 則評論,https://www.google.com/maps/place/%E6%BC%A2%E5...
3,RINGOAL車庫餐廳,383 則評論,https://www.google.com/maps/place/RINGOAL%E8%B...
4,橄欖樹廚房,669 則評論,https://www.google.com/maps/place/%E6%A9%84%E6...
...,...,...,...
285,夠麻吉股份有限公司,409 則評論,https://www.google.com/maps/place/%E5%A4%A0%E9...
286,紓咖,20 則評論,https://www.google.com/maps/place/%E7%B4%93%E5...
287,太魯閣晶英酒店梅園中餐廳,174 則評論,https://www.google.com/maps/place/%E5%A4%AA%E9...
288,CITYLINK 內湖店,"1,953 則評論",https://www.google.com/maps/place/CITYLINK+%E5...


### Bookstores

In [10]:
bookstore_after_df, records, remove_list = pd.read_csv("bookstore_after.csv", encoding = "utf-16"), set(), list()

for i in range(bookstore_after_df.shape[0]):
    if bookstore_after_df["Title"][i] in records:
        remove_list.append(i)
    else:
        records.add(bookstore_after_df["Title"][i])
        
bookstore_after_df.drop(labels = remove_list, axis = 0, inplace = True)
bookstore_after_df.to_csv("bookstore_after.csv", encoding = "utf-16", index = False)

bookstore_after_df

,Title,Comments,Url
0,麗文校園書局(交通大學店),68 則評論,https://www.google.com/maps/place/%E9%BA%97%E6...
1,水木書苑,763 則評論,https://www.google.com/maps/place/%E6%B0%B4%E6...
2,大學書局,"1,231 則評論",https://www.google.com/maps/place/%E5%A4%A7%E5...
3,筆耕小書店,86 則評論,https://www.google.com/maps/place/%E7%AD%86%E8...
4,校園書房(新竹勝利店),110 則評論,https://www.google.com/maps/place/%E6%A0%A1%E5...
...,...,...,...
201,書屋花甲Ｘ而立書店（台大店）,84 則評論,https://www.google.com/maps/place/%E6%9B%B8%E5...
202,太平青鳥,520 則評論,https://www.google.com/maps/place/%E5%A4%AA%E5...
203,書屋花甲,284 則評論,https://www.google.com/maps/place/%E6%9B%B8%E5...
204,練習曲書店,342 則評論,https://www.google.com/maps/place/%E7%B7%B4%E7...


### Departments

In [11]:
department_after_df, records, remove_list = pd.read_csv("department_after.csv", encoding = "utf-16"), set(), list()

for i in range(department_after_df.shape[0]):
    if department_after_df["Title"][i] in records:
        remove_list.append(i)
    else:
        records.add(department_after_df["Title"][i])
        
department_after_df.drop(labels = remove_list, axis = 0, inplace = True)
department_after_df.to_csv("department_after.csv", encoding = "utf-16", index = False)

department_after_df

,Title,Comments,Url
0,大聯大百貨生活館(交大門市),56 則評論,https://www.google.com/maps/place/%E5%A4%A7%E8...
1,小雅10元百貨,79 則評論,https://www.google.com/maps/place/%E5%B0%8F%E9...
2,水木百貨,103 則評論,https://www.google.com/maps/place/%E6%B0%B4%E6...
4,大信五金百貨複合館,34 則評論,https://www.google.com/maps/place/%E5%A4%A7%E4...
5,時代生活百貨,26 則評論,https://www.google.com/maps/place/%E6%99%82%E4...
...,...,...,...
155,德安百貨,"5,860 則評論",https://www.google.com/maps/place/%E5%BE%B7%E5...
156,DAISO大創百貨 台南德安店,479 則評論,https://www.google.com/maps/place/DAISO%E5%A4%...
157,漢神巨蛋購物廣場,"32,720 則評論",https://www.google.com/maps/place/%E6%BC%A2%E7...
158,漢神百貨,"8,876 則評論",https://www.google.com/maps/place/%E6%BC%A2%E7...


### Nightmarkets

In [12]:
nightmarket_after_df, records, remove_list = pd.read_csv("nightmarket_after.csv", encoding = "utf-16"), set(), list()

for i in range(nightmarket_after_df.shape[0]):
    if nightmarket_after_df["Title"][i] in records:
        remove_list.append(i)
    else:
        records.add(nightmarket_after_df["Title"][i])
        
nightmarket_after_df.drop(labels = remove_list, axis = 0, inplace = True)
nightmarket_after_df.to_csv("nightmarket_after.csv", encoding = "utf-16", index = False)

nightmarket_after_df

,Title,Comments,Url
0,新竹慈雲夜市,"1,346 則評論",https://www.google.com/maps/place/%E6%96%B0%E7...
1,青草湖夜市,762 則評論,https://www.google.com/maps/place/%E9%9D%92%E8...
2,竹中夜市,73 則評論,https://www.google.com/maps/place/%E7%AB%B9%E4...
3,新竹市觀光夜市協會,2 則評論,https://www.google.com/maps/place/%E6%96%B0%E7...
4,新竹市城隍廟夜市,"9,105 則評論",https://www.google.com/maps/place/%E6%96%B0%E7...
...,...,...,...
111,新夜市鴨肉麵,51 則評論,https://www.google.com/maps/place/%E6%96%B0%E5...
112,嘉義文化路夜市,"22,108 則評論",https://www.google.com/maps/place/%E5%98%89%E7...
113,士林鹽水雞,66 則評論,https://www.google.com/maps/place/%E5%A3%AB%E6...
114,彌陀夜市,"2,228 則評論",https://www.google.com/maps/place/%E5%BD%8C%E9...


### Outlets

In [13]:
outlet_after_df, records, remove_list = pd.read_csv("outlet_after.csv", encoding = "utf-16"), set(), list()

for i in range(outlet_after_df.shape[0]):
    if outlet_after_df["Title"][i] in records:
        remove_list.append(i)
    else:
        records.add(outlet_after_df["Title"][i])
        
outlet_after_df.drop(labels = remove_list, axis = 0, inplace = True)
outlet_after_df.to_csv("outlet_after.csv", encoding = "utf-16", index = False)

outlet_after_df

,Title,Comments,Url
0,Timberland 新竹愛買 Outlet,22 則評論,https://www.google.com/maps/place/Timberland+%...
1,GLORIA OUTLETS華泰名品城,"36,364 則評論",https://www.google.com/maps/place/GLORIA+OUTLE...
2,MITSUI OUTLET PARK 林口,"38,888 則評論",https://www.google.com/maps/place/MITSUI+OUTLE...
3,麗寶Outlet Mall,"21,671 則評論",https://www.google.com/maps/place/%E9%BA%97%E5...
4,UNDER ARMOUR 林口三井OUTLET店,"2,211 則評論",https://www.google.com/maps/place/UNDER+ARMOUR...
...,...,...,...
179,御殿場PREMIUM OUTLETS,"20,135 則評論",https://www.google.com/maps/place/%E5%BE%A1%E6...
180,三井OUTLET PARK 木更津,"12,904 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...
181,SHIATZY CHEN 夏姿陳,4 則評論,https://www.google.com/maps/place/SHIATZY+CHEN...
182,三井OUTLET PARK 幕張,"7,749 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...


## Combination

### Before

In [14]:
department_after_df = pd.read_csv("department_after.csv", encoding = "utf-16")
nightmarket_after_df = pd.read_csv("nightmarket_after.csv", encoding = "utf-16")
outlet_after_df = pd.read_csv("outlet_after.csv", encoding = "utf-16")

entertainment_df = pd.concat([department_after_df, nightmarket_after_df, outlet_after_df])
entertainment_df.to_csv("entertainment.csv", encoding = "utf-16", index = False)

entertainment_df

,Title,Comments,Url
0,大聯大百貨生活館(交大門市),56 則評論,https://www.google.com/maps/place/%E5%A4%A7%E8...
1,小雅10元百貨,79 則評論,https://www.google.com/maps/place/%E5%B0%8F%E9...
2,水木百貨,103 則評論,https://www.google.com/maps/place/%E6%B0%B4%E6...
3,大信五金百貨複合館,34 則評論,https://www.google.com/maps/place/%E5%A4%A7%E4...
4,時代生活百貨,26 則評論,https://www.google.com/maps/place/%E6%99%82%E4...
...,...,...,...
178,御殿場PREMIUM OUTLETS,"20,135 則評論",https://www.google.com/maps/place/%E5%BE%A1%E6...
179,三井OUTLET PARK 木更津,"12,904 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...
180,SHIATZY CHEN 夏姿陳,4 則評論,https://www.google.com/maps/place/SHIATZY+CHEN...
181,三井OUTLET PARK 幕張,"7,749 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...


### After

In [15]:
entertainment_df, remove_list = pd.read_csv("entertainment.csv", encoding = "utf-16"), list()

for i in range(entertainment_df.shape[0]):
    comments = entertainment_df["Comments"][i]
    comments = int(comments[0:comments.find("則評論")].replace(',', ''))
    if comments <= 25: remove_list.append(i)

entertainment_df.drop(remove_list, axis = 0, inplace = True)
entertainment_df.to_csv("entertainment_after.csv", encoding = "utf-16", index = False)

entertainment_df

,Title,Comments,Url
0,大聯大百貨生活館(交大門市),56 則評論,https://www.google.com/maps/place/%E5%A4%A7%E8...
1,小雅10元百貨,79 則評論,https://www.google.com/maps/place/%E5%B0%8F%E9...
2,水木百貨,103 則評論,https://www.google.com/maps/place/%E6%B0%B4%E6...
3,大信五金百貨複合館,34 則評論,https://www.google.com/maps/place/%E5%A4%A7%E4...
4,時代生活百貨,26 則評論,https://www.google.com/maps/place/%E6%99%82%E4...
...,...,...,...
441,寶島鐘錶 林口三井店,107 則評論,https://www.google.com/maps/place/%E5%AF%B6%E5...
443,御殿場PREMIUM OUTLETS,"20,135 則評論",https://www.google.com/maps/place/%E5%BE%A1%E6...
444,三井OUTLET PARK 木更津,"12,904 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...
446,三井OUTLET PARK 幕張,"7,749 則評論",https://www.google.com/maps/place/%E4%B8%89%E4...


## Check

### Before

In [16]:
attraction_after_df = pd.read_csv("attraction_after.csv", encoding = "utf-16")
restaurant_after_df = pd.read_csv("restaurant_after.csv", encoding = "utf-16")
bookstore_after_df = pd.read_csv("bookstore_after.csv", encoding = "utf-16")
entertainment_after_df = pd.read_csv("entertainment_after.csv", encoding = "utf-16")

In [17]:
records, remove_list = set(), list()

for i in range(entertainment_after_df.shape[0]):
    records.add(entertainment_after_df["Title"][i])
    
for i in range(attraction_after_df.shape[0]):
    if attraction_after_df["Title"][i] in records:
        print("Attraction:", attraction_after_df["Title"][i])
        remove_list.append(("attraction_after", i))
    else:
        records.add(entertainment_after_df["Title"][i])
    
for i in range(restaurant_after_df.shape[0]):
    if restaurant_after_df["Title"][i] in records:
        print("Restuarant:", restaurant_after_df["Title"][i])
        remove_list.append(("restaurant_after", i))
    else:
        records.add(restaurant_after_df["Title"][i])
    
for i in range(bookstore_after_df.shape[0]):
    if bookstore_after_df["Title"][i] in records:
        print("Bookstore:", bookstore_after_df["Title"][i])
        remove_list.append(("bookstore_after", i))
    else:
        records.add(bookstore_after_df["Title"][i])

Attraction:  新竹市城隍廟夜市  
Attraction:  桃園觀光夜市  
Attraction:  嘉義文化路夜市  
Attraction:  樹林頭觀光夜市  
Restuarant:  山口刺身ま丼飯專賣清大店#新竹美食#清大夜市美食#日本料理推薦#在地推薦美食#新竹必吃美食  
Restuarant:  Big City遠東巨城購物中心  
Restuarant:  SOGO 百貨新竹店  
Restuarant:  遠東SOGO 台北忠孝館  
Bookstore:  禾園茶館  


### Attractions

In [18]:
attraction_list = [index for name, index in remove_list if name == "attraction_after"]

attraction_after_df.drop(attraction_list, axis = 0, inplace = True)
attraction_after_df.to_csv("attraction_after.csv", encoding = "utf-16", index = False)

attraction_after_df

,Title,Comments,Url
0,竹湖,31 則評論,https://www.google.com/maps/place/%E7%AB%B9%E6...
1,寄梅亭,10 則評論,https://www.google.com/maps/place/%E5%AF%84%E6...
2,清華大學成功湖,379 則評論,https://www.google.com/maps/place/%E6%B8%85%E8...
3,靜心湖,"2,107 則評論",https://www.google.com/maps/place/%E9%9D%9C%E5...
4,沃夫帕克WolfPark 親子館(新竹親子餐廳/親子景點/室內親子),529 則評論,https://www.google.com/maps/place/%E6%B2%83%E5...
...,...,...,...
289,好好聚落,395 則評論,https://www.google.com/maps/place/%E5%A5%BD%E5...
290,宮原眼科,"24,400 則評論",https://www.google.com/maps/place/%E5%AE%AE%E5...
291,鳳鼻頭文化遺址,200 則評論,https://www.google.com/maps/place/%E9%B3%B3%E9...
292,高雄市忠烈祠,"1,896 則評論",https://www.google.com/maps/place/%E9%AB%98%E9...


### Restaurants

In [19]:
restaurant_list = [index for name, index in remove_list if name == "restaurant_after"]

restaurant_after_df.drop(restaurant_list, axis = 0, inplace = True)
restaurant_after_df.to_csv("restaurant_after.csv", encoding = "utf-16", index = False)

restaurant_after_df

,Title,Comments,Url
0,LALA Kitchen 新美式餐廳 交大店,994 則評論,https://www.google.com/maps/place/LALA+Kitchen...
1,和選旅-硯宴軒,350 則評論,https://www.google.com/maps/place/%E5%92%8C%E9...
2,漢堡王Burger King 交大店,315 則評論,https://www.google.com/maps/place/%E6%BC%A2%E5...
3,RINGOAL車庫餐廳,383 則評論,https://www.google.com/maps/place/RINGOAL%E8%B...
4,橄欖樹廚房,669 則評論,https://www.google.com/maps/place/%E6%A9%84%E6...
...,...,...,...
269,夠麻吉股份有限公司,409 則評論,https://www.google.com/maps/place/%E5%A4%A0%E9...
270,紓咖,20 則評論,https://www.google.com/maps/place/%E7%B4%93%E5...
271,太魯閣晶英酒店梅園中餐廳,174 則評論,https://www.google.com/maps/place/%E5%A4%AA%E9...
272,CITYLINK 內湖店,"1,953 則評論",https://www.google.com/maps/place/CITYLINK+%E5...


### Bookstores

In [20]:
bookstore_list = [index for name, index in remove_list if name == "bookstore_after"]

bookstore_after_df.drop(bookstore_list, axis = 0, inplace = True)
bookstore_after_df.to_csv("bookstore_after.csv", encoding = "utf-16", index = False)

bookstore_after_df

,Title,Comments,Url
0,麗文校園書局(交通大學店),68 則評論,https://www.google.com/maps/place/%E9%BA%97%E6...
1,水木書苑,763 則評論,https://www.google.com/maps/place/%E6%B0%B4%E6...
2,大學書局,"1,231 則評論",https://www.google.com/maps/place/%E5%A4%A7%E5...
3,筆耕小書店,86 則評論,https://www.google.com/maps/place/%E7%AD%86%E8...
4,校園書房(新竹勝利店),110 則評論,https://www.google.com/maps/place/%E6%A0%A1%E5...
...,...,...,...
198,書屋花甲Ｘ而立書店（台大店）,84 則評論,https://www.google.com/maps/place/%E6%9B%B8%E5...
199,太平青鳥,520 則評論,https://www.google.com/maps/place/%E5%A4%AA%E5...
200,書屋花甲,284 則評論,https://www.google.com/maps/place/%E6%9B%B8%E5...
201,練習曲書店,342 則評論,https://www.google.com/maps/place/%E7%B7%B4%E7...


### After

In [21]:
attraction_after_df = pd.read_csv("attraction_after.csv", encoding = "utf-16")
restaurant_after_df = pd.read_csv("restaurant_after.csv", encoding = "utf-16")
bookstore_after_df = pd.read_csv("bookstore_after.csv", encoding = "utf-16")
entertainment_after_df = pd.read_csv("entertainment_after.csv", encoding = "utf-16")

In [22]:
records, remove_list = set(), list()

for i in range(entertainment_after_df.shape[0]):
    records.add(entertainment_after_df["Title"][i])
    
for i in range(attraction_after_df.shape[0]):
    if attraction_after_df["Title"][i] in records:
        print("Attraction:", attraction_after_df["Title"][i])
        remove_list.append(("attraction_after", i))
    else:
        records.add(entertainment_after_df["Title"][i])
    
for i in range(restaurant_after_df.shape[0]):
    if restaurant_after_df["Title"][i] in records:
        print("Restuarant:", restaurant_after_df["Title"][i])
        remove_list.append(("restaurant_after", i))
    else:
        records.add(restaurant_after_df["Title"][i])
    
for i in range(bookstore_after_df.shape[0]):
    if bookstore_after_df["Title"][i] in records:
        print("Bookstore:", bookstore_after_df["Title"][i])
        remove_list.append(("bookstore_after", i))
    else:
        records.add(bookstore_after_df["Title"][i])